## Importação das bibliotecas

In [1]:
import pandas as pd
import locale
import re

## Visualização inicial do DataFrame

In [35]:
# Caminho do arquivo Excel
excel_file = "dados\Export_Vendas_Sintetico.xlsx"

# Leitura do arquivo Excel e seleção das colunas desejadas
df = pd.read_excel(excel_file, sheet_name='Planilha1', usecols=['Data', 'Valor', 'Status', 'Operador', 'Vendedor'])

# Exibir as primeiras linhas do DataFrame
print(df.head())

# Exibir os tipos de dados ajustados
print(df.dtypes)

                  Data  Valor     Status Operador Vendedor
0  21/12/2018 08:28:57  10.35  CANCELADA   Amanda     Neto
1  21/12/2018 09:16:04  10.35  CANCELADA     João   Victor
2  21/12/2018 09:16:02  10.35  CANCELADA     Taís    Lucas
3  21/12/2018 09:19:16   9.31  CANCELADA  Marcelo  Regiane
4  21/12/2018 09:23:14 224.85    REGULAR   Thales   Rebeca
Data         object
Valor       float64
Status       object
Operador     object
Vendedor     object
dtype: object


## Ajuste dos dados

In [23]:
# Renomear as colunas
df.rename(columns={
    'Data': 'data',
    'Valor': 'valor',
    'Status': 'status',
    'Operador': 'operador',
    'Vendedor': 'vendedor'
}, inplace=True)




# Definir o locale para Português do Brasil (pt_BR)
locale.setlocale(locale.LC_NUMERIC, 'pt_BR.UTF-8')

# Função para converter valores
def convert_to_float(value):
    if isinstance(value, str):
        value = value.replace('.', '').replace(',', '.')
    return float(value)

# Aplicar a função de conversão na coluna 'valor' para float aplicando
df['valor'] = df['valor'].apply(convert_to_float)

# Converter a coluna 'data' para Timestamp
try:
    df['data'] = pd.to_datetime(df['data'])
except ValueError as e:
    print(f"Não foi possível converter a coluna 'data' para Timestamp: {e}")
    
# Truncar a coluna 'status' para 10 caracteres
df['status'] = df['status'].apply(lambda x: x[:10])

# Exibir as primeiras linhas do DataFrame após a conversão
print(df.head())

# Exibir os tipos de dados ajustados
print(df.dtypes)

                 data    valor     status operador vendedor
0 2018-12-21 08:28:57   10.350  CANCELADA   Amanda     Neto
1 2018-12-21 09:16:04   10.350  CANCELADA     João   Victor
2 2018-12-21 09:16:02   10.350  CANCELADA     Taís    Lucas
3 2018-12-21 09:19:16    9.315  CANCELADA  Marcelo  Regiane
4 2018-12-21 09:23:14  224.850    REGULAR   Thales   Rebeca
data        datetime64[ns]
valor              float64
status              object
operador            object
vendedor            object
dtype: object


## Checagem dos Dados

In [25]:
# visualizar o DataFrame

# Ajustar as opções de exibição do pandas
pd.options.display.float_format = '{:.2f}'.format

# Exibir o DataFrame completo
df

,data,valor,status,operador,vendedor
0,2018-12-21 08:28:57,10.35,CANCELADA,Amanda,Neto
1,2018-12-21 09:16:04,10.35,CANCELADA,João,Victor
2,2018-12-21 09:16:02,10.35,CANCELADA,Taís,Lucas
3,2018-12-21 09:19:16,9.31,CANCELADA,Marcelo,Regiane
4,2018-12-21 09:23:14,224.85,REGULAR,Thales,Rebeca
...,...,...,...,...,...
103202,2023-01-04 16:55:52,294.60,REGULAR,Thales,Victor
103203,2023-01-04 16:58:35,359.97,REGULAR,Amanda,Neto
103204,2023-01-04 17:00:39,539.96,REGULAR,João,Victor
103205,2023-01-04 17:01:40,179.98,REGULAR,Taís,Lucas


In [26]:
# número de linhas e colunas
df.shape

(103207, 5)

In [27]:
# nome das colunas
df.columns

Index(['data', 'valor', 'status', 'operador', 'vendedor'], dtype='object')

In [28]:
# informações gerais
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103207 entries, 0 to 103206
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   data      103207 non-null  datetime64[ns]
 1   valor     103207 non-null  float64       
 2   status    103207 non-null  object        
 3   operador  103207 non-null  object        
 4   vendedor  103207 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 3.9+ MB


In [29]:
# checagem de informações vazias
df.isnull().sum()

data        0
valor       0
status      0
operador    0
vendedor    0
dtype: int64

In [31]:
# dados estatísticos
with pd.option_context("float_format", "{:.2f}".format):
    display(df.describe())

,valor
count,103207.00
mean,276.49
std,386.53
min,0.00
25%,148.50
50%,224.85
75%,344.13
max,58534.23


In [33]:
df.nunique()

data        103030
valor        11818
status           3
operador         5
vendedor         6
dtype: int64

In [34]:
df.duplicated().sum()

0